# Amazon Bedrock Batch Inference

## 情報
- https://docs.aws.amazon.com/bedrock/latest/userguide/batch-inference.html
- 2024.1.25
    - Public Preview
    - 利用方法
        - REST API: あると思うが面倒
        - CLI: 無さそう
        - SDK: プレビューのがある
            - PythonとJavaのみ
            - https://d2eo22ngex1n9g.cloudfront.net/Documentation/SDK/bedrock-python-sdk-reinvent.zip
        - コンソール: 無さそう

## 検証内容
- Bedrockのbatch推論のquotaがon demandと関係ない（だからquotaに引っ掛からなくてうれしい）と言う説
- text to image (SDXL or Titan or Both)で500枚くらいバッチ推論ジョブを発行してどんな風に実行が完了するか（もしくは完了しないか）を見てみる
- On demandのquotaより明らかに早かったら嬉しい